In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


import numpyro
import numpyro.distributions as dist
from numpyro import handlers
from numpyro.infer import MCMC, NUTS

import jax
import jax.numpy as jnp
from jax import random, vmap
from jax.scipy.special import logsumexp
from jax import lax

In [7]:
data_path = '../data/processed/data_processed.csv'
data = pd.read_csv(data_path)

# Use data only for specific machineID
machine_id = 1
data_1 = data.loc[data.machineID==machine_id]

In [15]:
# Change type of datetime column to datetime (the object type)
data_1['datetime'] = pd.to_datetime(data_1.datetime)

/tmp/ipykernel_32069/777938672.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_1['datetime'] = pd.to_datetime(data_1.datetime)


In [17]:
# Drop the previously used index column named Unnamed: 0
data_1.drop(['Unnamed: 0'], axis=1, inplace=True)

/tmp/ipykernel_32069/665843931.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_1.drop(['Unnamed: 0'], axis=1, inplace=True)


In [19]:
data_1.columns

Index(['machineID', 'datetime', 'voltmean_3h', 'rotatemean_3h',
       'pressuremean_3h', 'vibrationmean_3h', 'voltsd_3h', 'rotatesd_3h',
       'pressuresd_3h', 'vibrationsd_3h', 'voltmean_24h', 'rotatemean_24h',
       'pressuremean_24h', 'vibrationmean_24h', 'voltsd_24h', 'rotatesd_24h',
       'pressuresd_24h', 'vibrationsd_24h', 'error1count', 'error2count',
       'error3count', 'error4count', 'error5count', 'comp1', 'comp2', 'comp3',
       'comp4', 'comp1_life', 'comp2_life', 'comp3_life', 'comp4_life', 'age',
       'model_model1', 'model_model2', 'model_model3', 'model_model4',
       'failure', 'comp1_fail', 'comp2_fail', 'comp3_fail', 'comp4_fail'],
      dtype='object')

We can also create 2 new features, the $sin$ and $cos$ of the (also created) hour feature.  

For now, in the testing, I will use the telemetry and error count data and have as output ($y$) a specific component's remaining life.

In [21]:
input_columns = ['voltmean_3h', 'rotatemean_3h',
       'pressuremean_3h', 'vibrationmean_3h', 'voltsd_3h', 'rotatesd_3h',
       'pressuresd_3h', 'vibrationsd_3h', 'voltmean_24h', 'rotatemean_24h',
       'pressuremean_24h', 'vibrationmean_24h', 'voltsd_24h', 'rotatesd_24h',
       'pressuresd_24h', 'vibrationsd_24h', 'error1count', 'error2count',
       'error3count', 'error4count', 'error5count','model_model1', 'model_model2', 'model_model3', 'model_model4']

output_columns = ['comp1_life']

In [22]:
X_df = data_1[input_columns]
y_df = data_1[output_columns]

In [27]:
X = X_df.values
y = y_df.values

It is not a good idea to randomly split the dataset to train and split, because the temporal model relies on the continuity of the input data. As a result, the split is performed as taking the first $n$ percent of the available data as training set, while the rest is used as a test set:

In [33]:
train_perc = 0.8 # percentage of data used as training set

X_train = X[:int(train_perc*X.shape[0])]
y_train = y[:int(train_perc*X.shape[0])]

X_test = X[int(train_perc*X.shape[0]):]
y_test = y[int(train_perc*X.shape[0]):]

In [36]:
n_train, p = X_train.shape
n_test, _ = X_test.shape

Create the model:

In [43]:
def f(carry, noise_t):
    beta, z_prev, tau = carry
    z_t = beta*z_prev + noise_t
    z_prev = z_t
    return (beta, z_prev, tau), z_t

In [44]:
def model(T, T_forecast, obs1=None, ix_mis1=None, ix_obs1=None, obs2=None, ix_mis2=None, ix_obs2=None):
    """
    Define priors over beta, tau, sigma, z_1 (keep the shapes in mind)
    """
    beta = numpyro.sample(name="beta", fn=dist.Normal(loc=jnp.zeros(2), scale=jnp.ones(2)))
    tau = numpyro.sample(name="tau", fn=dist.HalfCauchy(scale=jnp.ones(2)))
    sigma = numpyro.sample(name="sigma", fn=dist.HalfCauchy(scale=.1))
    z_prev = numpyro.sample(name="z_1", fn=dist.Normal(loc=jnp.zeros(2), scale=jnp.ones(2)))
    
    """
    Define LKJ prior
    """
    L_Omega = numpyro.sample("L_Omega", dist.LKJCholesky(2, 10.))
    Sigma_lower = jnp.matmul(jnp.diag(jnp.sqrt(tau)), L_Omega) # lower cholesky factor of the covariance matrix
    noises = numpyro.sample("noises", fn=dist.MultivariateNormal(loc=jnp.zeros(2), scale_tril=Sigma_lower), sample_shape=(T+T_forecast-1,))
    
    """
    Propagate the dynamics forward using jax.lax.scan
    """
    carry = (beta, z_prev, tau)
    z_collection = [z_prev]
    carry, zs_exp = lax.scan(f, carry, noises, T+T_forecast-1)
    z_collection = jnp.concatenate((jnp.array(z_collection), zs_exp), axis=0)
    
    """
    Sample the observed y (y_obs) and missing y (y_mis)
    """
    numpyro.sample(name="y_mis1", fn=dist.Normal(loc=z_collection[ix_mis1, 0], scale=sigma), obs=None)
    numpyro.sample(name="y_obs1", fn=dist.Normal(loc=z_collection[ix_obs1, 0], scale=sigma), obs=obs1)
    numpyro.sample(name="y_mis2", fn=dist.Normal(loc=z_collection[ix_mis2, 1], scale=sigma), obs=None)
    numpyro.sample(name="y_obs2", fn=dist.Normal(loc=z_collection[ix_obs2, 1], scale=sigma), obs=obs2)
    return z_collection

In [46]:
rng_key = random.PRNGKey(0)
rng_key, rng_key_ = random.split(rng_key)

nuts_kernel = NUTS(model=model)
mcmc = MCMC(nuts_kernel, num_samples=1000, num_warmup=1000, num_chains=1)
mcmc.run(rng_key_, T=N, T_forecast=0, obs1=y_obs1, ix_mis1=ix_mis1, ix_obs1=ix_obs1, 
         obs2=y_obs2, ix_mis2=ix_mis2, ix_obs2=ix_obs2)

NameError: name 'N' is not defined